<a href="https://colab.research.google.com/github/ujjalkumarmaity/NLP/blob/main/NER/Named_Entity_Recognition_with_BERT_wikiann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer,BertForTokenClassification
from pprint import pprint


In [3]:
dataset = load_dataset('wikiann','en')

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print(dataset.shape)
print(dataset['train'])
print(dataset['train'][4])

{'validation': (10000, 4), 'test': (10000, 4), 'train': (20000, 4)}
Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 20000
})
{'tokens': ['Her', 'daughter', 'from', 'the', 'second', 'marriage', 'was', 'Marie', "d'Agoult", '(', '1805–1876', ')', ',', 'who', 'in', 'turn', 'gave', 'birth', 'to', 'several', 'children', ',', 'among', 'them—from', 'her', 'liaison', 'to', 'Franz', 'Liszt', '–-', 'Cosima', 'Wagner', '(', '1837–1930', ')', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0], 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], 'spans': ["PER: Marie d'Agoult", 'PER: Franz Liszt', 'PER: Cosima Wagner']}


In [10]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [11]:
def tokenization(x):
    return tokenizer(x['tokens'],padding='max_length',truncation=True,is_split_into_words=True)
tok_data = dataset.map(tokenization)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [12]:
# Here input_ids length and ner_tags length not same
len(tok_data['train'][0]['input_ids']),len(tok_data['train'][0]['ner_tags'])

(512, 11)

In [46]:
pprint(dataset["train"][0]['tokens'])
pprint(tokenizer(dataset["train"][0]['tokens'])['input_ids'])

['R.H.',
 'Saunders',
 '(',
 'St.',
 'Lawrence',
 'River',
 ')',
 '(',
 '968',
 'MW',
 ')']
[[101, 1054, 1012, 1044, 1012, 102],
 [101, 15247, 102],
 [101, 1006, 102],
 [101, 2358, 1012, 102],
 [101, 5623, 102],
 [101, 2314, 102],
 [101, 1007, 102],
 [101, 1006, 102],
 [101, 5986, 2620, 102],
 [101, 12464, 102],
 [101, 1007, 102]]


In [38]:
z = tokenizer.batch_encode_plus(dataset['train']["tokens"], is_split_into_words=True, truncation=True)
print(z.word_ids(batch_index=0))
print(z['input_ids'][0])

[None, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]
[101, 1054, 1012, 1044, 1012, 15247, 1006, 2358, 1012, 5623, 2314, 1007, 1006, 5986, 2620, 12464, 1007, 102]


In [40]:
def adjusted_tokenized_label(data):
    tokenized_data = tokenizer.batch_encode_plus(data["tokens"], is_split_into_words=True, truncation=True)
    tokenized_label = []
    for ind in range(len(tokenized_data['input_ids'])):
        prev_id = -1
        word_id = tokenized_data.word_ids(batch_index=ind)
        ner_label =  data['ner_tags'][ind]
        temp = []
        i = -1
        for w in word_id:
            if w is None:
                temp.append(-100)
            elif w==prev_id:
                temp.append(ner_label[i])
            else:
                i += 1
                temp.append(ner_label[i])
                prev_id = w
        tokenized_label.append(temp)
    tokenized_data['label'] = tokenized_label
    return tokenized_data

In [42]:
tokenized_data = dataset.map(adjusted_tokenized_label,batched=True,remove_columns=['tokens', 'ner_tags', 'langs', 'spans'] )

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]